In [1]:
from pyspark.sql import SparkSession
from operator import add
import re

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("A3_AN_Spark")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 8)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/21 21:40:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# read a file from local filesystem oif your driver
fileSV = spark_context.textFile('hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.sv')
fileEN = spark_context.textFile('hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.en')
linesSV = fileSV.count()
linesEN = fileEN.count()
print(f"Lines SV: {linesSV}, Lines EN: {linesEN}")

24/02/21 21:40:48 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 221.5 KiB, free 434.2 MiB)
24/02/21 21:40:48 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 32.6 KiB, free 434.2 MiB)
24/02/21 21:40:48 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-170-de1:10005 (size: 32.6 KiB, free: 434.4 MiB)
24/02/21 21:40:48 INFO SparkContext: Created broadcast 0 from textFile at NativeMethodAccessorImpl.java:0
24/02/21 21:40:48 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 221.5 KiB, free 433.9 MiB)
24/02/21 21:40:48 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 32.6 KiB, free 433.9 MiB)
24/02/21 21:40:48 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on host-192-168-2-170-de1:10005 (size: 32.6 KiB, free: 434.3 MiB)
24/02/21 21:40:48 INFO SparkContext: Created broadcast 1 from textFile at NativeMethodAccessorImpl.java:

Lines SV: 1862234, Lines EN: 1862234


24/02/21 21:40:55 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 3) in 830 ms on 192.168.2.254 (executor 0) (2/2)
24/02/21 21:40:55 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/02/21 21:40:55 INFO DAGScheduler: ResultStage 1 (count at /tmp/ipykernel_18179/3277269576.py:5) finished in 0.841 s
24/02/21 21:40:55 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/21 21:40:55 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
24/02/21 21:40:55 INFO DAGScheduler: Job 1 finished: count at /tmp/ipykernel_18179/3277269576.py:5, took 0.849882 s


In [3]:
numPartitions = fileSV.getNumPartitions()
print(numPartitions)

3


In [4]:
lowercaseSV = fileSV.map(lambda x: x.lower())
lowercaseEN = fileEN.map(lambda x: x.lower())
tokenSV = lowercaseSV.map(lambda x: x.split())
tokenEN = lowercaseEN.map(lambda x: x.split())
#regex = re.compile(r"\b\w+(?:'\w+)?\b")
#tokenSV = lowercaseSV.flatMap(lambda x: regex.findall(x))
#tokenEN = lowercaseEN.flatMap(lambda x: regex.findall(x))
#tokenSV = lowercaseSV.flatMap(lambda x: re.sub(r'[^\w\s]', '', x).split())
#tokenEN = lowercaseEN.flatMap(lambda x: re.sub(r'[^\w\s]', '', x).split())


In [5]:
verifyLinesSV = tokenSV.count()
verifyLinesEN = tokenEN.count()
print(verifyLinesSV)
print(verifyLinesEN)

24/02/21 21:40:55 INFO SparkContext: Starting job: count at /tmp/ipykernel_18179/1381452307.py:1
24/02/21 21:40:55 INFO DAGScheduler: Got job 2 (count at /tmp/ipykernel_18179/1381452307.py:1) with 3 output partitions
24/02/21 21:40:55 INFO DAGScheduler: Final stage: ResultStage 2 (count at /tmp/ipykernel_18179/1381452307.py:1)
24/02/21 21:40:55 INFO DAGScheduler: Parents of final stage: List()
24/02/21 21:40:55 INFO DAGScheduler: Missing parents: List()
24/02/21 21:40:55 INFO DAGScheduler: Submitting ResultStage 2 (PythonRDD[6] at count at /tmp/ipykernel_18179/1381452307.py:1), which has no missing parents
24/02/21 21:40:55 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 9.9 KiB, free 433.9 MiB)
24/02/21 21:40:55 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 5.8 KiB, free 433.9 MiB)
24/02/21 21:40:55 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on host-192-168-2-170-de1:10005 (size: 5.8 KiB, free: 434.3 

1862234
1862234


24/02/21 21:41:01 INFO TaskSetManager: Finished task 0.0 in stage 3.0 (TID 8) in 2334 ms on 192.168.2.254 (executor 0) (1/2)
24/02/21 21:41:01 INFO TaskSetManager: Finished task 1.0 in stage 3.0 (TID 9) in 2340 ms on 192.168.2.254 (executor 0) (2/2)
24/02/21 21:41:01 INFO TaskSchedulerImpl: Removed TaskSet 3.0, whose tasks have all completed, from pool 
24/02/21 21:41:01 INFO DAGScheduler: ResultStage 3 (count at /tmp/ipykernel_18179/1381452307.py:2) finished in 2.358 s
24/02/21 21:41:01 INFO DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/21 21:41:01 INFO TaskSchedulerImpl: Killing all running tasks in stage 3: Stage finished
24/02/21 21:41:01 INFO DAGScheduler: Job 3 finished: count at /tmp/ipykernel_18179/1381452307.py:2, took 2.363198 s


In [6]:
reTokenSV = lowercaseSV.flatMap(lambda x: re.findall(r"\b\w+\b", x))
reTokenEN = lowercaseEN.flatMap(lambda x: re.findall(r"\b\w+\b", x))
nWordsSV = reTokenSV.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
nWordsEN = reTokenEN.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
tenMostCommonSV = nWordsSV.sortBy(lambda x: x[1], ascending=False).take(10)
tenMostCommonEN = nWordsEN.sortBy(lambda x: x[1], ascending=False).take(10)
print(tenMostCommonSV)
print(tenMostCommonEN)

24/02/21 21:41:01 INFO SparkContext: Starting job: sortBy at /tmp/ipykernel_18179/291135609.py:5
24/02/21 21:41:01 INFO DAGScheduler: Registering RDD 9 (reduceByKey at /tmp/ipykernel_18179/291135609.py:3) as input to shuffle 0
24/02/21 21:41:01 INFO DAGScheduler: Got job 4 (sortBy at /tmp/ipykernel_18179/291135609.py:5) with 3 output partitions
24/02/21 21:41:01 INFO DAGScheduler: Final stage: ResultStage 5 (sortBy at /tmp/ipykernel_18179/291135609.py:5)
24/02/21 21:41:01 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 4)
24/02/21 21:41:01 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 4)
24/02/21 21:41:01 INFO DAGScheduler: Submitting ShuffleMapStage 4 (PairwiseRDD[9] at reduceByKey at /tmp/ipykernel_18179/291135609.py:3), which has no missing parents
24/02/21 21:41:01 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 13.1 KiB, free 433.9 MiB)
24/02/21 21:41:01 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (

[('att', 1709969), ('och', 1351079), ('i', 1054764), ('det', 953045), ('som', 917621), ('för', 915166), ('av', 740770), ('är', 701851), ('en', 636939), ('vi', 546126)]
[('the', 3506102), ('of', 1662891), ('to', 1545280), ('and', 1320053), ('in', 1099408), ('that', 839196), ('a', 776918), ('is', 774969), ('for', 538487), ('we', 526580)]


24/02/21 21:41:42 INFO BlockManagerInfo: Removed broadcast_13_piece0 on host-192-168-2-170-de1:10005 in memory (size: 6.5 KiB, free: 434.3 MiB)
24/02/21 21:41:42 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on 192.168.2.254:10005 (size: 5.9 KiB, free: 434.3 MiB)
24/02/21 21:41:42 INFO BlockManagerInfo: Removed broadcast_13_piece0 on 192.168.2.254:10005 in memory (size: 6.5 KiB, free: 434.3 MiB)
24/02/21 21:41:42 INFO MapOutputTrackerMasterEndpoint: Asked to send map output locations for shuffle 3 to 192.168.2.254:54258
24/02/21 21:41:42 INFO TaskSetManager: Finished task 0.0 in stage 17.0 (TID 31) in 77 ms on 192.168.2.254 (executor 0) (1/1)
24/02/21 21:41:42 INFO TaskSchedulerImpl: Removed TaskSet 17.0, whose tasks have all completed, from pool 
24/02/21 21:41:42 INFO DAGScheduler: ResultStage 17 (runJob at PythonRDD.scala:181) finished in 0.088 s
24/02/21 21:41:42 INFO DAGScheduler: Job 9 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/2

In [7]:
# The top ten most common words generated reflect what is the most commond words in reality as well
print("# The top ten most common words generated reflect what is the most commond words in reality as well")

# The top ten most common words generated reflect what is the most commond words in reality as well


In [10]:
#print(tokenSV.take(5))
sv_1 = tokenSV.flatMap(lambda x: x)
en_1 = tokenEN.flatMap(lambda x: x)
sv_1 = sv_1.map(lambda x: ' '.join(x))
en_1 = en_1.map(lambda x: ' '.join(x))
sv_1 = sv_1.zipWithIndex()
en_1 = en_1.zipWithIndex()
sv_2 = sv_1.map(lambda x: (x[1], [0]))
en_2 = en_1.map(lambda x: (x[1], [0]))
joined_3 = sv_2.join(en_2)
filter_4 = joined_3.filter(lambda x: x[1][0] and x[1][1])
filter_5 = filter_4.filter(lambda x: len(x[1][0].split()) <= 8 and len(x[1][1].split()) <= 8)
filter_6 = filter_5.filter(lambda x: len(x[1][0].split()) == len(x[1][1].split()))
map_7 = filter_6.map(lambda x: (zip(x[1][0].split(), x[1][1].split())))
reduce_8 = map_7.flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)


SyntaxError: invalid syntax (3650069181.py, line 17)

24/02/21 22:10:47 INFO BlockManagerInfo: Removed broadcast_17_piece0 on host-192-168-2-170-de1:10005 in memory (size: 5.4 KiB, free: 434.3 MiB)
24/02/21 22:10:47 INFO BlockManagerInfo: Removed broadcast_17_piece0 on 192.168.2.254:10005 in memory (size: 5.4 KiB, free: 434.3 MiB)
